In [1]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np

from CWScopeTesting import *
from WPI_SCA_LIBRARY.FileFormat import *
from WPI_SCA_LIBRARY.Metrics import *

In [2]:
traces = np.load("trace_set_10k.npy")
numOfTraces=10
print('Number of traces:', numOfTraces)
numOfSamples=len(traces[0])
print('Number of samples:', numOfSamples)

plaintext = np.zeros(shape=(numOfTraces, 16))

filename="plaintext_new.txt"
with open(filename) as f:
    content = f.readlines()
content = [x.strip() for x in content]
plaintext = np.array([bytearray.fromhex(c) for c in content])



Number of traces: 10
Number of samples: 15000


In [3]:
#traces
#numOfTraces
#numOfSamples
#plaintext

#lets create a file
os.remove('demoFileFormat.hdf5')
fileClass = HDF5FileClass("demoFileFormat.hdf5")

#creating an experiment
fileClass.addExperiment("demoExperiment", definition="Demo Holding Data")
experimentOne = fileClass.experiments["demoExperiment"]

#creating datasets within the experiment
experimentOne.addDataset("plaintext", (numOfTraces, 16), definition="Plaintext Input To the Algorithem", dtype='uint8')
plaintextDataset = experimentOne.dataset["plaintext"]

experimentOne.addDataset("traces", (numOfTraces, numOfSamples), definition="Traces from the Oscilliscope", dtype='float64')
tracesDataset = experimentOne.dataset["traces"]

experimentOne.addDataset("Labels", (numOfTraces - 1, 1), definition="Labels!!!", dtype='uint8')
labels = experimentOne.dataset["Labels"]

#lets write to the dataset
for i in range(numOfTraces):
        plaintextDataset.addData(i, plaintext[i])
        
tracesDataset.addData(range(numOfTraces), traces[range(numOfTraces)])
for i in range(9):
        labels.addData(i, random.randint(0,1))

In [4]:
#re open file and take a look at the experiments
with h5py.File('demoFileFormat.hdf5', "a") as data_file:
    fileClass = HDF5FileClass('demoFileFormat.hdf5',fileInputType="existing", data_file=data_file)
    
    #get experiments
    print(fileClass.getExperimentDefinitions())
    

{'demoExperiment': 'Demo Holding Data'}


In [5]:
#re open file and take a look at the datasets
with h5py.File('demoFileFormat.hdf5', "a") as data_file:
    fileClass = HDF5FileClass('demoFileFormat.hdf5',fileInputType="existing", data_file=data_file)
    
    #get experiments
    demoExperiment = fileClass.experiments['demoExperiment']
    
    #view definitions
    print(demoExperiment.getDatasetDefinitions())

{'Labels': 'Labels!!!', 'plaintext': 'Plaintext Input To the Algorithem', 'traces': 'Traces from the Oscilliscope'}


In [6]:
with h5py.File('demoFileFormat.hdf5', "a") as data_file:
    fileClass = HDF5FileClass('demoFileFormat.hdf5',fileInputType="existing", data_file=data_file)
    
    #get experiments
    demoExperiment = fileClass.experiments['demoExperiment']
    
    #get labels dataset class
    label = demoExperiment.dataset['Labels']
    
    #rename labels
    label.rename('labelA', demoExperiment)
    
    #redefine labels
    demoExperiment.redefine('labelA', "Randomly Generated Label Value")
    
    #resize labels
    print(labels.getDatasetSize())
    demoExperiment.resizeDataset('labelA', [10,1])
    print(labels.getDatasetSize())
    
    label.addData(9,1)

(9, 1)
(10, 1)


In [7]:
#re test results
with h5py.File('demoFileFormat.hdf5', "a") as data_file:
    fileClass = HDF5FileClass('demoFileFormat.hdf5',fileInputType="existing", data_file=data_file)
    
    #get experiments
    demoExperiment = fileClass.experiments['demoExperiment']
    
    #view definitions
    print(demoExperiment.getDatasetDefinitions())

{'labelA': 'Randomly Generated Label Value', 'plaintext': 'Plaintext Input To the Algorithem', 'traces': 'Traces from the Oscilliscope'}


In [9]:
#metadata
with h5py.File('demoFileFormat.hdf5', "a") as data_file:
    fileClass = HDF5FileClass('demoFileFormat.hdf5',fileInputType="existing", data_file=data_file)
    
    #get experiments
    demoExperiment = fileClass.experiments['demoExperiment']
    
    #read meta data
    print(demoExperiment.readMetadata())
    
    #add meta data
    demoExperiment.addMetadata('Crypto Algorithem','AES 128')
    
    print(demoExperiment.readMetadata())

{'labelA_dataset_description': 'Randomly Generated Label Value', 'plaintext_dataset_description': 'Plaintext Input To the Algorithem', 'traces_dataset_description': 'Traces from the Oscilliscope'}
{'labelA_dataset_description': 'Randomly Generated Label Value', 'plaintext_dataset_description': 'Plaintext Input To the Algorithem', 'traces_dataset_description': 'Traces from the Oscilliscope', 'Crypto Algorithem': 'AES 128'}


In [10]:
#still exists 
with h5py.File('demoFileFormat.hdf5', "a") as data_file:
    fileClass = HDF5FileClass('demoFileFormat.hdf5',fileInputType="existing", data_file=data_file)
    
    #get experiments
    demoExperiment = fileClass.experiments['demoExperiment']
    
    print(demoExperiment.readMetadata())

{'Crypto Algorithem': 'AES 128', 'labelA_dataset_description': 'Randomly Generated Label Value', 'plaintext_dataset_description': 'Plaintext Input To the Algorithem', 'traces_dataset_description': 'Traces from the Oscilliscope'}


In [19]:
#still exists 
with h5py.File('demoFileFormat.hdf5', "a") as data_file:
    fileClass = HDF5FileClass('demoFileFormat.hdf5',fileInputType="existing", data_file=data_file)
    
    #get experiments
    demoExperiment = fileClass.experiments['demoExperiment']
    
    traces = demoExperiment.dataset['traces']
    
    print("Single Read")
    print(traces.readData(1))
    
    print("Double Read")
    print(traces.readData([1,2,3,4]))
    
    print("Read All")
    readAllKeys, readAllValues = demoExperiment.readAll([1])
    print(readAllKeys)
    print(readAllValues)
    
    print("Value From Read All")
    print(readAllValues[readAllKeys['traces']])
    
    print("Get Specified Dataset")
    specificDataSets = demoExperiment.getSpecifiedDatasets(['traces', 'labelA'],1)
    print(specificDataSets)

Single Read
[-0.01093884 -0.0085948  -0.00586009 ... -0.00468807 -0.0042974
 -0.00351605]
Double Read
[[-0.01093884 -0.0085948  -0.00586009 ... -0.00468807 -0.0042974
  -0.00351605]
 [-0.00937614 -0.00742278 -0.00468807 ... -0.00507874 -0.00546942
  -0.0042974 ]
 [-0.01054816 -0.00937614 -0.00664144 ... -0.00468807 -0.00468807
  -0.00468807]
 [-0.01093884 -0.00898547 -0.00586009 ... -0.0042974  -0.0042974
  -0.00390673]]
Read All
{'labelA': 0, 'plaintext': 1, 'traces': 2}
[array([[0]], dtype=uint8), array([[ 10, 128,   8, 191,  52, 166, 201,   4, 159,  57, 137, 213,  64,
        248, 142, 119]], dtype=uint8), array([[-0.01093884, -0.0085948 , -0.00586009, ..., -0.00468807,
        -0.0042974 , -0.00351605]])]
Value From Read All
[[-0.01093884 -0.0085948  -0.00586009 ... -0.00468807 -0.0042974
  -0.00351605]]
Get Specified Dataset
({'traces': 0, 'labelA': 1}, [array([-0.01093884, -0.0085948 , -0.00586009, ..., -0.00468807,
       -0.0042974 , -0.00351605]), array([0], dtype=uint8)])
